# The contribution of small RNAs to the evolution of separate sexes and sex chromosomes in the plant _Silene latifolia_

#### Eddy Mendoza-Galindo, Aline Muyle, CEFE Montpellier
This notebook contains all the necessary code for the paper published in the Journal of Evolutionary Biology

Correspondence: aline.muyle[at]cefe.cnrs.fr

Open for academic and research purposes only, 2025

# <span style="color:#4dd98b"> Alingment and quantification

In [5]:
! cat scripts/count.sh

cd raw/sRNA_MGX/trimmed/
for file in *.fastq
do
echo "working with $file"
perl -e ' $count=0; $len=0; while(<>) { s/\r?\n//; s/\t/ /g; if (s/^@//) { if ($. != 1) { print "\n" } s/ |$/\t/; $count++; $_ .= "\t"; } else { s/ //g; $len += length($_) } print $_; } print "\n"; ' $file | sed -E 's/^.+\t(\w+)\+.*$/\1/g' | perl -e ' $col=0; while (<>) { s/\r?\n//; @F = split /\t/, $_; $len = length($F[$col]); print "$_\t$len\n" }; ' | awk '$2 ~ /(21|22|24)/ ' > ${file}_count.tsv
done


In [6]:
! bash scripts/count.sh

working with F1B_final_trimming.fastq
working with F1L_final_trimming.fastq
working with F2B_final_trimming.fastq
working with F2L_final_trimming.fastq
working with F3B_final_trimming.fastq
working with F3L_final_trimming.fastq
working with M1B_final_trimming.fastq
working with M1L_final_trimming.fastq
working with M2B_final_trimming.fastq
working with M2L_final_trimming.fastq
working with M4B_final_trimming.fastq
working with M4L_final_trimming.fastq


### <span style="color:#4dd98b"> Alingment, females without Y

In [2]:
# Remove Y chromosome from genome and map Females
! grep ">" genome/genome3.fa | grep -v "scaffold" | sed 's/>//' > genome/no_y.list
! seqtk subseq genome/genome3.fa genome/no_y.list > genome/no_y.fa
! grep ">" genome/no_y.fa

>chr1
>chr2
>chr3
>chr4
>chr5
>chr6
>chr7
>chr8
>chr9
>chr10
>chr11
>chr12


In [8]:
# create a list file of the bams
! ls females_no_y/F*.bam > bam.list
! ls males/M*.bam >> bam.list

### <span style="color:#4dd98b">Merge females (without y) and males (with y) (Inside the conda ShortStack4 environment so it works)

samtools merge -r -@ 12 -f -o merged.bam -b bam.list
    
#### <span style="color:#4dd98b"> First, PTGS (21-22 nt), asking for _de novo_ and template-based miRNA annotation
    
ShortStack --genomefile genome/genome3.fa --bamfile merged.bam --threads 8 --outdir only_21-22_known_de_novo_f-y --dicermax 22 --mmap u --dn_mirna --knownRNAs ../raw/caryophyllaceae_mirnas.fa 

#### <span style="color:#4dd98b"> Then, RDdM (24 nt), no miRNA identification
    
ShortStack --genomefile genome/genome3.fa --bamfile merged.bam --threads 8 --outdir only_24_f-y --dicermin 23 --mmap u 

### <span style="color:#4dd98b"> Depth Flowers

In [1]:
# get bam for each lenght and individual
! rm -r raw/bams_per_length/
! mkdir raw/bams_per_length

# first 21-22, females
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/F1B_ptgs.bam females_no_y/F1B_dicer.bam 
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/F2B_ptgs.bam females_no_y/F2B_dicer.bam
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/F3B_ptgs.bam females_no_y/F3B_dicer.bam 

# then 24, females
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/F1B_rddm.bam females_no_y/F1B_dicer.bam 
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/F2B_rddm.bam females_no_y/F2B_dicer.bam
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/F3B_rddm.bam females_no_y/F3B_dicer.bam 

# first 21-22, males
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/M1B_ptgs.bam males/M1B_dicer.bam
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/M2B_ptgs.bam males/M2B_dicer.bam
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/M4B_ptgs.bam males/M4B_dicer.bam

# then 24, males
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/M1B_rddm.bam males/M1B_dicer.bam 
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/M2B_rddm.bam males/M2B_dicer.bam
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/M4B_rddm.bam males/M4B_dicer.bam

In [2]:
# calculate depth for each individual, only for flower buds and convert it to bed in flowers
# females
! samtools depth -@ 12 raw/bams_per_length/F*B_ptgs.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/females_flowers_ptgs.bed
! samtools depth -@ 12 raw/bams_per_length/F*B_rddm.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/females_flowers_rddm.bed
# males
! samtools depth -@ 12 raw/bams_per_length/M*B_ptgs.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/males_flowers_ptgs.bed
! samtools depth -@ 12 raw/bams_per_length/M*B_rddm.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/males_flowers_rddm.bed

In [5]:
# get the number of mapped reads for the normalization 
! echo "____________________ FEMALES FLOWERS_____________________"
! samtools view -c -F 260 -@ 12 females_no_y/F1B_dicer.bam
! samtools view -c -F 260 -@ 12 females_no_y/F2B_dicer.bam 
! samtools view -c -F 260 -@ 12 females_no_y/F3B_dicer.bam 
! echo "____________________ MALES FLOWERS_____________________"
! samtools view -c -F 260 -@ 12 males/M1B_dicer.bam
! samtools view -c -F 260 -@ 12 males/M2B_dicer.bam
! samtools view -c -F 260 -@ 12 males/M4B_dicer.bam

____________________ FEMALES FLOWERS_____________________
27241167
29660632
25216360
____________________ MALES FLOWERS_____________________
13200497
23914694
23578008


In [ ]:
# Get depths for each gene
# 21/22

# Females
! bedtools intersect -a raw/depth/females_flowers_ptgs.bed -b annotation/mrnas.bed -wb | cut -f 1-6,10 > raw/depth/flowers_females_gene_depth_ptgs.tsv
# Males
! bedtools intersect -a raw/depth/males_flowers_ptgs.bed -b annotation/mrnas.bed -wb | cut -f 1-6,10 > raw/depth/flowers_males_gene_depth_ptgs.tsv

# 24

# Add promoter
! bedtools slop -i annotation/mrnas.bed -g chromSizes.txt -l 200 -r 0 -s > annotation/mrnas_plus_promoter.bed

# Females
! bedtools intersect -a raw/depth/females_flowers_rddm.bed -b annotation/mrnas_plus_promoter.bed -wb | cut -f 1-6,10 > raw/depth/flowers_females_gene_depth_rddm.tsv
# Males
! bedtools intersect -a raw/depth/males_flowers_rddm.bed -b annotation/mrnas_plus_promoter.bed -wb | cut -f 1-6,10 > raw/depth/flowers_males_gene_depth_rddm.tsv

### <span style="color:#4dd98b"> Depth Leaves

In [11]:
# first 21-22, females
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/F1L_ptgs.bam females_no_y/F1L_dicer.bam 
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/F2L_ptgs.bam females_no_y/F2L_dicer.bam
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/F3L_ptgs.bam females_no_y/F3L_dicer.bam 

# then 24, females
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/F1L_rddm.bam females_no_y/F1L_dicer.bam 
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/F2L_rddm.bam females_no_y/F2L_dicer.bam
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/F3L_rddm.bam females_no_y/F3L_dicer.bam 

# first 21-22, males
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/M1L_ptgs.bam males/M1L_dicer.bam
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/M2L_ptgs.bam males/M2L_dicer.bam
! samtools view  -h -e 'length(seq)==21 || length(seq)==22' -@ 12 -o raw/bams_per_length/M4L_ptgs.bam males/M4L_dicer.bam

# then 24, males
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/M1L_rddm.bam males/M1L_dicer.bam 
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/M2L_rddm.bam males/M2L_dicer.bam
! samtools view  -h -e 'length(seq)==24' -@ 12 -o raw/bams_per_length/M4L_rddm.bam males/M4L_dicer.bam

In [12]:
# Calculate depth for leaves

# females
! samtools depth -@ 12 raw/bams_per_length/F*L_ptgs.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/females_leaves_ptgs.bed
! samtools depth -@ 12 raw/bams_per_length/F*L_rddm.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/females_leaves_rddm.bed
# males
! samtools depth -@ 12 raw/bams_per_length/M*L_ptgs.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/males_leaves_ptgs.bed
! samtools depth -@ 12 raw/bams_per_length/M*L_rddm.bam | awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5}' > raw/depth/males_leaves_rddm.bed

In [5]:
# get the number of mapped reads for the normalization 
! echo "____________________ FEMALES LEAVES_____________________"
! samtools view -c -F 260 -@ 12 females_no_y/F1L_dicer.bam
! samtools view -c -F 260 -@ 12 females_no_y/F2L_dicer.bam 
! samtools view -c -F 260 -@ 12 females_no_y/F3L_dicer.bam 
! echo "____________________ MALES LEAVES_____________________"
! samtools view -c -F 260 -@ 12 males/M1L_dicer.bam
! samtools view -c -F 260 -@ 12 males/M2L_dicer.bam
! samtools view -c -F 260 -@ 12 males/M4L_dicer.bam

____________________ FEMALES LEAVES_____________________
24695806
31425418
21443081
____________________ MALES LEAVES_____________________
20502566
17763390
26371650


In [ ]:
# Get depths for each gene
# 21/22

# Females
! bedtools intersect -a raw/depth/females_leaves_ptgs.bed -b annotation/mrnas.bed -wb | cut -f 1-6,10 > raw/depth/leaves_females_gene_depth_ptgs.tsv
# Males
! bedtools intersect -a raw/depth/males_leaves_ptgs.bed -b annotation/mrnas.bed -wb | cut -f 1-6,10 > raw/depth/leaves_males_gene_depth_ptgs.tsv

# 24 (including promoter)

# Females
! bedtools intersect -a raw/depth/females_leaves_rddm.bed -b annotation/mrnas_plus_promoter.bed -wb | cut -f 1-6,10 > raw/depth/leaves_females_gene_depth_rddm.tsv
# Males
! bedtools intersect -a raw/depth/males_leaves_rddm.bed -b annotation/mrnas_plus_promoter.bed -wb | cut -f 1-6,10 > raw/depth/leaves_males_gene_depth_rddm.tsv

### <span style="color:#4dd98b"> For the Circos Plot

In [ ]:
# due to an error, we need to update the alingment with a V4 genome
# to download remotely

# get new chromosome sizes
samtools faidx genome/silati_v4.fa
cat genome/silati_v4.fa.fai | cut -f 1-2 > v4_chromSizes.txt

In [ ]:
# download new repeat annotation
gff2bed < annotation/v4_repeats.gff > annotation/v4_repeats.bed

rm annotation/v4_repeats.gff

In [ ]:
# update bed file to v4 version

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain annotation/mrnas.bed annotation/v4_mrnas.bed

In [ ]:
# update sRNA mapping BED files with the new genome using crossmap
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/females_flowers_rddm.bed raw/depth/v4_females_flowers_rddm.bed

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/males_flowers_rddm.bed raw/depth/v4_males_flowers_rddm.bed

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/females_leaves_rddm.bed raw/depth/v4_females_leaves_rddm.bed

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/males_leaves_rddm.bed raw/depth/v4_males_leaves_rddm.bed

In [ ]:
# somehow we need to rename the files cause chrY is now Y
sed 's/Y/chrY/g' raw/depth/v4_females_flowers_rddm.bed > raw/depth/females_flowers_rddm.bed
sed 's/Y/chrY/g' raw/depth/v4_males_flowers_rddm.bed > raw/depth/males_flowers_rddm.bed

sed 's/Y/chrY/g' raw/depth/v4_females_leaves_rddm.bed > raw/depth/females_leaves_rddm.bed
sed 's/Y/chrY/g' raw/depth/v4_males_leaves_rddm.bed > raw/depth/males_leaves_rddm.bed

rm raw/depth/v4*

In [ ]:
# remotely
bedtools makewindows -g v4_chromSizes.txt -w 1000000 > annotation/1mb_windows.bed

# flowers depth
bedtools intersect -b annotation/1mb_windows.bed -a raw/depth/females_flowers_rddm.bed -wb -wa | awk '{print $0 "\t" $7 "_" $8 "_" $9}' > raw/depth/females_flowers_rddm.depth
bedtools intersect -b annotation/1mb_windows.bed -a raw/depth/males_flowers_rddm.bed -wb -wa | awk '{print $0 "\t" $7 "_" $8 "_" $9}' > raw/depth/males_flowers_rddm.depth

# leaves depth
bedtools intersect -b annotation/1mb_windows.bed -a raw/depth/females_leaves_rddm.bed -wb -wa | awk '{print $0 "\t" $7 "_" $8 "_" $9}' > raw/depth/females_leaves_rddm.depth
bedtools intersect -b annotation/1mb_windows.bed -a raw/depth/males_leaves_rddm.bed -wb -wa | awk '{print $0 "\t" $7 "_" $8 "_" $9}' > raw/depth/males_leaves_rddm.depth

In [ ]:
# Rscript for the table for the paper

library(data.table)
library(tidyverse)

# function
surco = function(toy, lib_size_1, lib_size_2, lib_size_3) {
          # set column names
          colnames(toy) = c("chr", "start", "end", "depth_1", "depth_2", "depth_3", "chr_w", "start_w", "end_w", "wind")
          # calculate mead depth per window, normalize by library size
          toy[, mean_depth_1 := mean(depth_1)/lib_size_1, by=.(wind)]
          toy[, mean_depth_2 := mean(depth_2)/lib_size_2, by=.(wind)]
          toy[, mean_depth_3 := mean(depth_3)/lib_size_3, by=.(wind)]
          # get one row per window
          toy = unique(toy, by = "wind")[, .(chr_w, start_w, end_w, wind, mean_depth_1, mean_depth_2, mean_depth_3)]
          # overall mean and multiply by 1m
          toy[, mean_depth := (mean_depth_1 + mean_depth_2 + mean_depth_3)*(1000000/3), by=1:nrow(toy)]
          toy[, log_depth := log(mean_depth), by=1:nrow(toy)]
          # print bed file + mean depth per million
          toy[, .(chr_w, start_w, end_w, mean_depth)]
}

### Flowers

# Females
flo_w_fe_d = fread("raw/depth/females_flowers_rddm.depth")
flo_w_fe_d = surco(flo_w_fe_d, 27241167, 29660632, 25216360)
flo_w_fe_d = flo_w_fe_d %>% mutate(tissue = "flowers", sex = "female")

# Males
flo_w_ma_d = fread("raw/depth/males_flowers_rddm.depth")
flo_w_ma_d = surco(flo_w_ma_d, 13200497, 23914694, 23578008)
flo_w_ma_d = flo_w_ma_d %>% mutate(tissue = "flowers", sex = "male")

#### Leaves

# Females
le_w_fe_d = fread("raw/depth/females_leaves_rddm.depth")
le_w_fe_d = surco(le_w_fe_d, 24695806, 31425418, 21443081)
le_w_fe_d = le_w_fe_d %>% mutate(tissue = "leaves", sex = "female")

# Males
le_w_ma_d = fread("raw/depth/males_leaves_rddm.depth")
le_w_ma_d = surco(le_w_ma_d, 20502566, 17763390, 26371650)
le_w_ma_d = le_w_ma_d %>% mutate(tissue = "leaves", sex = "male")

# concatenate and save

wind_map = rbind(flo_w_fe_d, flo_w_ma_d, le_w_fe_d, le_w_ma_d)
write.table(wind_map, file = "results/1mb_windows_mapping.tsv", quote = F, sep = "\t", col.names = T, row.names = F)

In [ ]:
# remove windows data bc it is too heavy
rm raw/depth/*.depth

#### 1MB TPM for Circos

In [ ]:
# create the windows, avoid overlapping windows and add a window id
! bedtools makewindows -g v4_chr_sizes.txt -s 1000000 -w 999999 -i srcwinnum > depth/v4_1mb_windows.bed 

# create the intersections
#### 21-22 nt
! bedtools intersect -nonamecheck -a females_flowers_ptgs.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_females_flowers_ptgs.tsv
! bedtools intersect -nonamecheck -a males_flowers_ptgs.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_males_flowers_ptgs.tsv

! bedtools intersect -nonamecheck -a females_leaves_ptgs.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_females_leaves_ptgs.tsv
! bedtools intersect -nonamecheck -a males_leaves_ptgs.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_males_leaves_ptgs.tsv

##### 24 nt
! bedtools intersect -nonamecheck -a females_flowers_rddm.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_females_flowers_rddm.tsv
! bedtools intersect -nonamecheck -a males_flowers_rddm.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_males_flowers_rddm.tsv

! bedtools intersect -nonamecheck -a females_leaves_rddm.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_females_leaves_rddm.tsv
! bedtools intersect -nonamecheck -a males_leaves_rddm.bed -b depth/v4_1mb_windows.bed -wb | cut -f 1-6,10 > depth/1mb_males_leaves_rddm.tsv

## R Surco Script - one for TPM, one for raw counts

The same was done for the 1Mb windows

In [ ]:
#!/bin/bash
# Name: lats
# Author: Eddy Mendoza
# Date: 29/01/23

# === SLURM Config ===
#SBATCH -J surco
#SBATCH --get-user-env
#SBATCH --partition=batch
#SBATCH --time=08:00:00
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=16
#SBATCH --cpus-per-task=5
#SBATCH --mem=16G
#SBATCH -o scripts/exegfiles/surco.out
#SBATCH -e scripts/exegfiles/surco.err

# === SET UP ENVIRONMENT ===

cd taller/lats/
ml R

# === WORK COMMANDS ===
Rscript surco_genes.R

In [ ]:
#!/usr/bin/env Rscript

library(data.table)
library(tidyverse)
library(reshape2)

# function
surco = function(toy, lib_size_1, lib_size_2, lib_size_3) {

          # set column names
          colnames(toy) = c("chr", "start", "end", "depth_1", "depth_2", "depth_3", "gene")

          # calculate mead depth per window, normalize by library size
          toy[, indv_1 := mean(depth_1)*1000000000/lib_size_1, by=.(gene)]
          toy[, indv_2 := mean(depth_2)*1000000000/lib_size_2, by=.(gene)]
          toy[, indv_3 := mean(depth_3)*1000000000/lib_size_3, by=.(gene)]

          # get one row per gene
          toy = unique(toy, by = "gene")[, .(gene, indv_1, indv_2, indv_3)]

          # print
          toy
}

########################################## RDDM ################################################################################################################

########## Flowers ##############

# Females
flo_w_fe_d = fread("depth/females_flowers_rddm.tsv")
flo_w_fe_d = surco(flo_w_fe_d, 27241167, 29660632, 25216360)
flo_w_fe_d = flo_w_fe_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "flowers", sex = "female", path = "rddm")

# Males
flo_w_ma_d = fread("depth/males_flowers_rddm.tsv")
flo_w_ma_d = surco(flo_w_ma_d, 13200497, 23914694, 23578008)
flo_w_ma_d = flo_w_ma_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "flowers", sex = "male", path = "rddm")

########### Leaves ##############

# Females
le_w_fe_d = fread("depth/females_leaves_rddm.tsv")
le_w_fe_d = surco(le_w_fe_d, 24695806, 31425418, 21443081)
le_w_fe_d = le_w_fe_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "leaves", sex = "female", path = "rddm")

# Males
le_w_ma_d = fread("depth/males_leaves_rddm.tsv")
le_w_ma_d = surco(le_w_ma_d, 20502566, 17763390, 26371650)
le_w_ma_d = le_w_ma_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "leaves", sex = "male", path = "rddm")

########################################## PTGS ################################################################################################################

########## Flowers ##############

# Females
flo_w_fe_t = fread("depth/females_flowers_ptgs.tsv")
flo_w_fe_t = surco(flo_w_fe_t, 27241167, 29660632, 25216360)
flo_w_fe_t = flo_w_fe_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "flowers", sex = "female", path = "ptgs")

# Males
flo_w_ma_t = fread("depth/males_flowers_ptgs.tsv")
flo_w_ma_t = surco(flo_w_ma_t, 13200497, 23914694, 23578008)
flo_w_ma_t = flo_w_ma_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "flowers", sex = "male", path = "ptgs")

########### Leaves ##############

# Females
le_w_fe_t = fread("depth/females_leaves_ptgs.tsv")
le_w_fe_t = surco(le_w_fe_t, 24695806, 31425418, 21443081)
le_w_fe_t = le_w_fe_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "leaves", sex = "female", path = "ptgs")

# Males
le_w_ma_t = fread("depth/males_leaves_ptgs.tsv")
le_w_ma_t = surco(le_w_ma_t, 20502566, 17763390, 26371650)
le_w_ma_t = le_w_ma_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "TPM") %>% mutate(tissue = "leaves", sex = "male", path = "ptgs")

########################################## MERGE ################################################################################################################

# concatenate and save

gene_map = rbind(flo_w_fe_d, flo_w_ma_d, le_w_fe_d, le_w_ma_d, flo_w_fe_t, flo_w_ma_t, le_w_fe_t, le_w_ma_t)

write.table(gene_map, file = "depth/v4_tpm_gene_mapping_inds.tsv", quote = F, sep = "\t", col.names = T, row.names = F)

In [ ]:
#!/usr/bin/env Rscript

library(data.table)
library(tidyverse)
library(reshape2)

# function
surco = function(toy, lib_size_1, lib_size_2, lib_size_3) {

          # set column names
          colnames(toy) = c("chr", "start", "end", "depth_1", "depth_2", "depth_3", "gene")

          # calculate mead depth per window, normalize by library size
          toy[, indv_1 := sum(depth_1), by=.(gene)]
          toy[, indv_2 := sum(depth_2), by=.(gene)]
          toy[, indv_3 := sum(depth_3), by=.(gene)]

          # get one row per gene
          toy = unique(toy, by = "gene")[, .(gene, indv_1, indv_2, indv_3)]

          # print
          toy
}

########################################## RDDM ################################################################################################################

########## Flowers ##############

# Females
flo_w_fe_d = fread("depth/females_flowers_rddm.tsv")
flo_w_fe_d = surco(flo_w_fe_d, 27241167, 29660632, 25216360)
flo_w_fe_d = flo_w_fe_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "flowers", sex = "female", path = "rddm")

# Males
flo_w_ma_d = fread("depth/males_flowers_rddm.tsv")
flo_w_ma_d = surco(flo_w_ma_d, 13200497, 23914694, 23578008)
flo_w_ma_d = flo_w_ma_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "flowers", sex = "male", path = "rddm")

########### Leaves ##############

# Females
le_w_fe_d = fread("depth/females_leaves_rddm.tsv")
le_w_fe_d = surco(le_w_fe_d, 24695806, 31425418, 21443081)
le_w_fe_d = le_w_fe_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "leaves", sex = "female", path = "rddm")

# Males
le_w_ma_d = fread("depth/males_leaves_rddm.tsv")
le_w_ma_d = surco(le_w_ma_d, 20502566, 17763390, 26371650)
le_w_ma_d = le_w_ma_d %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "leaves", sex = "male", path = "rddm")

########################################## PTGS ################################################################################################################

########## Flowers ##############

# Females
flo_w_fe_t = fread("depth/females_flowers_ptgs.tsv")
flo_w_fe_t = surco(flo_w_fe_t, 27241167, 29660632, 25216360)
flo_w_fe_t = flo_w_fe_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "flowers", sex = "female", path = "ptgs")

# Males
flo_w_ma_t = fread("depth/males_flowers_ptgs.tsv")
flo_w_ma_t = surco(flo_w_ma_t, 13200497, 23914694, 23578008)
flo_w_ma_t = flo_w_ma_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "flowers", sex = "male", path = "ptgs")

########### Leaves ##############

# Females
le_w_fe_t = fread("depth/females_leaves_ptgs.tsv")
le_w_fe_t = surco(le_w_fe_t, 24695806, 31425418, 21443081)
le_w_fe_t = le_w_fe_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "leaves", sex = "female", path = "ptgs")

# Males
le_w_ma_t = fread("depth/males_leaves_ptgs.tsv")
le_w_ma_t = surco(le_w_ma_t, 20502566, 17763390, 26371650)
le_w_ma_t = le_w_ma_t %>% melt(id.vars = "gene", variable.name="individual", value.name = "counts") %>% mutate(tissue = "leaves", sex = "male", path = "ptgs")

########################################## MERGE ################################################################################################################

# concatenate and save

gene_map = rbind(flo_w_fe_d, flo_w_ma_d, le_w_fe_d, le_w_ma_d, flo_w_fe_t, flo_w_ma_t, le_w_fe_t, le_w_ma_t)

write.table(gene_map, file = "depth/v4_raw_gene_mapping_inds.tsv", quote = F, sep = "\t", col.names = T, row.names = F)

# <span style="color:#3de2d8"> RNA-seq analysis

In [ ]:
# get a bed file of the mRNAs (inside annotation/)
grep "mRNA" *.gff3 | awk '{print $1 "\t" $4 "\t" $5 "\t" $9 "\t" $6 "\t" $7}' | sed 's/ID=//g' | sed -E 's/;Parent.+=\w+//g' > mrnas.bed

# clean fasta
cat S.latifolia_v3.0_genes.mRNA.fna | sed -E 's/ CDS.+//g' > mrnas.fa 

# check if they match
grep -Fxv -f <(grep ">" mrnas.fa | sed 's/>//g' ) <(cut -f 4 mrnas.bed) |  wc -l

# 33084 female genes (without the Y scaffolds)
# get those genes

awk '$1 !~ /scaffold/' annotation/mrnas.bed > annotation/female_mrnas.bed

# get the sequences only for those
perl -e ' ($id,$fasta)=@ARGV; open(ID,$id); while (<ID>) { s/\r?\n//; /^>?(\S+)/; $ids{$1}++; } $num_ids = keys %ids; open(F, $fasta); $s_read = $s_wrote = $print_it = 0; while (<F>) { if (/^>(\S+)/) { $s_read++; if ($ids{$1}) { $s_wrote++; $print_it = 1; delete $ids{$1} } else { $print_it = 0 } }; if ($print_it) { print $_ } }; END { warn "Searched $s_read FASTA records.\nFound $s_wrote IDs out of $num_ids in the ID list.\n" } ' <(cut -f 4 annotation/female_mrnas.bed) annotation/mrnas.fa > annotation/female_mrnas.fa

In [5]:
# Build a Kallisto index
! kallisto index -i annotation/mrnas.idx annotation/mrnas.fa

# also for female genes
! kallisto index -i annotation/female_mrnas.idx annotation/female_mrnas.fa

# Four female plants (C1_26, C1_27, C1_29, C1_34) and four males (C1_01, C1_03, C1_04, C1_05)
# do the quantification of the females without the Y genes


[build] loading fasta file annotation/mrnas.fa
[build] k-mer length: 31
[build] warning: clipped off poly-A tail (longer than 10)
        from 27 target sequences
[build] warning: replaced 50 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 454447 contigs and contains 50244895 k-mers 



In [ ]:
# run kallisto
! bash ../scripts/kallisto_paper.sh # this was modified so I could do it separatelly for females and males

In [3]:
# Merge results
! ls rna_seq/kallisto/ | head -n 16 > rna_seq/kallisto/samples.list

# first I manually changed the dir names so it was less messy (move "combined" from the start to the end of the name)
mv kallisto/combined_C1_04_B kallisto/C1_04_B_combined
mv kallisto/combined_C1_05_B kallisto/C1_05_B_combined
mv kallisto/combined_C1_29_B kallisto/C1_29_B_combined
mv kallisto/combined_C1_34_B kallisto/C1_34_B_combined

# merge results
! bash ../scripts/merge_kallisto_raw_paper.sh 

C1_01_B RUNNING
C1_01_B DONE
_________
C1_01_L RUNNING
C1_01_L DONE
_________
C1_03_B RUNNING
C1_03_B DONE
_________
C1_03_L RUNNING
C1_03_L DONE
_________
C1_04_B_combined RUNNING
C1_04_B_combined DONE
_________
C1_04_L RUNNING
C1_04_L DONE
_________
C1_05_B_combined RUNNING
C1_05_B_combined DONE
_________
C1_05_L RUNNING
C1_05_L DONE
_________
C1_26_B RUNNING
C1_26_B DONE
_________
C1_26_L RUNNING
C1_26_L DONE
_________
C1_27_B RUNNING
C1_27_B DONE
_________
C1_27_L RUNNING
C1_27_L DONE
_________
C1_29_B_combined RUNNING
C1_29_B_combined DONE
_________
C1_29_L RUNNING
C1_29_L DONE
_________
C1_34_B_combined RUNNING
C1_34_B_combined DONE
_________
C1_34_L RUNNING
C1_34_L DONE
_________


# <span style="color:#3de2d8"> PTGS

In [1]:
! bedtools intersect  -nonamecheck -a tables/sex_biased_flowers_sbge.bed -b tables/sex_biased_flowers_ptgs.bed -wb | cut -f 4,8,13,18 | awk '$2 != $4'
! echo "____________________"
! bedtools intersect  -nonamecheck -a tables/sex_biased_leaves_sbge.bed -b tables/sex_biased_leaves_ptgs.bed -wb | cut -f 4,8,13,18 | awk '$2 != $4'

_chr5_000339.1	female-biased	Cluster_18098	male-biased
____________________


Only one candidate was found in flowers

In [34]:
# Get fasta sequences from the overlapping clusters, done remotely

awk 'FNR==NR{a[$0];next}{if (($2 in a)){print}}' <(cut -f 3 results/overlaps_flowers_ptgs.tsv | sort | uniq) only_21-22_known_de_novo_f-y/Results.txt | awk '{print $2 "\t" "21-22" "\t" $11}' > temp.tab

perl -e ' $len=0; while(<>) { s/\r?\n//; @F=split /\t/, $_; print ">$F[0]"; if (length($F[1])) { print " $F[1]" } print "\n"; $s=$F[2]; $len+= length($s); $s=~s/.{60}(?=.)/$&\n/g; print "$s\n"; } warn "\nConverted $. tab-delimited lines to FASTA format\nTotal sequence length: $len\n\n"; ' temp.tab > results/srnas_flowers.fa

rm temp.tab


Converted 12 tab-delimited lines to FASTA format
Total sequence length: 275



In [35]:
# Now for leaves

awk 'FNR==NR{a[$0];next}{if (($2 in a)){print}}' <(cut -f 3 results/overlaps_leaves_ptgs.tsv | sort | uniq) only_21-22_known_de_novo_f-y/Results.txt | awk '{print $2 "\t" "21-22" "\t" $11}' > temp.tab
perl -e ' $len=0; while(<>) { s/\r?\n//; @F=split /\t/, $_; print ">$F[0]"; if (length($F[1])) { print " $F[1]" } print "\n"; $s=$F[2]; $len+= length($s); $s=~s/.{60}(?=.)/$&\n/g; print "$s\n"; } warn "\nConverted $. tab-delimited lines to FASTA format\nTotal sequence length: $len\n\n"; ' temp.tab > results/srnas_leaves.fa

rm temp.tab


Converted 6 tab-delimited lines to FASTA format
Total sequence length: 141



#### <span style="color:#3de2d8">  Get inverted repeats from the genome (inside tes environment). Done remotely and using the v4 genome.

einverted -sequence genome/silati_v4.fa -threshold 80 -gap 8 -match 4 -mismatch -6 -maxrepeat 100 -outseq annotation/inv_rep.fa -outfile annotation/inv_rep.san
    
    Parameters used: [default]
    
    -gap penalty of 8 [12]
    -match score 4 [3]
    -mismatch score -6 [-4]
    -minimum score threshold 80 [50] (I guess it's calculated from the identity and the number of gaps -not documented)
    -maximum lenght from start to finish 100 (assuming at ~40bp in the terminal loop)

In [ ]:
# get the number of Inverted repeats
grep -c "Score" annotation/inv_rep.san

186404

In [ ]:
To create a bed file including the information about the repeat:
    
    - Chr
    - Start
    - End
    - ID
    - Score
    - Strand
    - Matching baisepairs
    - Repeat length
    - Mismatches
    - Number of gaps
    - Total mismatches (gaps + mismatches)

In [ ]:
# inside annotation/

paste \
<(grep "Score" inv_rep.san | sed -E 's/^(\w+):.+/\1/g' ) \
<(sed -n '/Score/ { n; p }' inv_rep.san | sed -E 's/\s*(\w+)\s*.+/\1/g' ) \
<(sed -n '/Score/ { n; n; n; p }' inv_rep.san | sed -E 's/\s*(\w+)\s*.+/\1/g' ) \
<(echo inverted_repeat_{1..186404} | sed -E 's/\s/\n/g' | awk '{print $0 "\t" "0" "\t" "+"}') \
<(grep "Score" inv_rep.san | sed -E 's/.+: (\w+)\/(\w+).+, (\w+) ga.+/\1\t\2\t\3/g' | awk '{print $1 "\t" $2 "\t" $2-$1 "\t" $3 "\t" $2-$1+$3 }' ) \
  > inv_rep.bed

In [ ]:
# Get fasta files for inverted repeats
! bedtools getfasta -nameOnly -fi genome/silati_v4.fa -bed annotation/inv_rep.bed -fo annotation/inv_rep_full.fa
# make it a BLAST database
! makeblastdb -in annotation/inv_rep_full.fa -dbtype nucl

In [108]:
# BLAST sRNAs against the inverted repeat library
# alingment above 15 bp
# reduced penalty for mismatches
# reduced gap penalty

# Flowers
blastn -db annotation/inv_rep_full.fa -query results/srnas_flowers.fa -num_threads 12 -task blastn-short -word_size 15 -penalty -1 -gapopen 1 -outfmt 6 -out results/srnas_flowers.blast 
# Leaves
blastn -db annotation/inv_rep_full.fa -query results/srnas_leaves.fa -num_threads 12 -task blastn-short -word_size 15 -penalty -1 -gapopen 1 -outfmt 6 -out results/srnas_leaves.blast

Only one inverted repeat matched with a cluster from flowers:

Cluster_41166   inverted_repeat_108651  86.957  23

Which is the same as: Cluster_41163_RdDM

gene: _chr12_004651.1

In [ ]:
# get annotations from the precursor and the cluster:

grep "inverted_repeat_108651" annotation/inv_rep.bed | cut -f 1-6 > results/precursor.bed

# precursor in chr 9 while cluster in chr X

In [ ]:
# to visualize the precursor:
http://rna.tbi.univie.ac.at//cgi-bin/RNAWebSuite/RNAfold.cgi
http://rna.tbi.univie.ac.at/forna/

In [ ]:
# check if the flowers' precursor is overlapping to a gene or a TE (done remotely)

bedtools intersect -a results/precursor.bed -b annotation/v4_mrnas.bed -wb

bedtools intersect -a results/precursor.bed -b annotation/v4_repeats.bed -wb

# Result:
chr9    108916489       108916681       .       1247EDTA     helitron        .       ID=TE_homo_1290913;Name=TE_00000471;Classification=DNA/Helitron;Sequence_ontology=SO:0000544;Identity=0.875;Method=homology

It matches with an helitron, therefore a heterochromatin siRNA

# <span style="color:#f0b27a"> RdDM

In [2]:
# flowers
! bedtools window -a tables/sex_biased_flowers_sbge.bed -b tables/sex_biased_flowers_rddm.bed -l 2200 -r 2000 | cut -f 4,8,13,17 | awk '$2 != $4'
! bedtools window -a tables/sex_biased_flowers_sbge.bed -b tables/sex_biased_flowers_rddm.bed -l 2200 -r 2000 | cut -f 4,8,13,17 | awk '$2 != $4' > tables/overlaps_flowers_rddm.tsv

! echo "##############################"

# leaves
! bedtools window -a tables/sex_biased_leaves_sbge.bed -b tables/sex_biased_leaves_rddm.bed -l 2200 -r 2000 | cut -f 4,8,13,17 | awk '$2 != $4'
! bedtools window -a tables/sex_biased_leaves_sbge.bed -b tables/sex_biased_leaves_rddm.bed -l 2200 -r 2000 | cut -f 4,8,13,17 | awk '$2 != $4' > tables/overlaps_leaves_rddm.tsv

_chr1_000597.1	male-biased	Cluster_1004_RdDM	female-biased
_chr10_001223.1	female-biased	Cluster_34088_RdDM	male-biased
_chr12_004651.1	male-biased	Cluster_41163_RdDM	female-biased
_chr12_003427.1	male-biased	Cluster_42967_RdDM	female-biased
_chr3_000060.1	male-biased	Cluster_10450_RdDM	female-biased
_chr3_000162.1	male-biased	Cluster_10628_RdDM	female-biased
_chr5_000339.1	female-biased	Cluster_18097_RdDM	male-biased
_chr7_001948.1	male-biased	Cluster_25601_RdDM	female-biased
_chr3_003800.1	male-biased	Cluster_10450_RdDM	female-biased
##############################
_chr9_001530.1	female-biased	Cluster_31262_RdDM	male-biased


## The coordinates of all the PTGS and RdDM clusters are the same!

# <span style="color:#18BBD2"> Evaluation of the sRNA candidates

In [ ]:
# The resunting bed files from the DE were updated with the V4 genome coordinates:

# Genes (remove last column or it doesn't work)
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain <(cut -f 1-8 results/differential_expression/sex_biased_flowers_sbge.bed) results/differential_expression/v4_sex_biased_flowers_sbge.bed
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain <(cut -f 1-8 results/differential_expression/sex_biased_leaves_sbge.bed) results/differential_expression/v4_sex_biased_leaves_sbge.bed

# PTGS
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain <(cut -f 1-9 results/differential_expression/sex_biased_flowers_ptgs.bed) results/differential_expression/v4_sex_biased_flowers_ptgs.bed
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain <(cut -f 1-9 results/differential_expression/sex_biased_leaves_ptgs.bed) results/differential_expression/v4_sex_biased_leaves_ptgs.bed

# RDDM (de una vez por si acaso)
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain <(cut -f 1-8 results/differential_expression/sex_biased_flowers_rddm.bed) results/differential_expression/v4_sex_biased_flowers_rddm.bed
CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain <(cut -f 1-8 results/differential_expression/sex_biased_leaves_rddm.bed) results/differential_expression/v4_sex_biased_leaves_rddm.bed

In [ ]:
#I validated (in the remote PC) all 9 overlaps (+1 in leaves) using the new coordinates. Then, I only extracted the bed files for the overlaps. Please note that the number of DE elements changed.

This because initially, I looked for the overlaps in my personal PC.

    awk 'FNR==NR{a[$0];next}{if (($4 in a)){print}}' <(cut -f 1 results/overlaps_flowers_rddm.tsv) annotation/v4_mrnas.bed > results/candidates_flowers_genes.bed
    
    awk 'FNR==NR{a[$0];next}{if (($4 in a)){print}}' <(cut -f 1 results/overlaps_leaves_rddm.tsv) annotation/v4_mrnas.bed > results/candidates_leaves_genes.bed

    awk 'FNR==NR{a[$0];next}{if (($4 in a)){print}}' <(cut -f 3 results/overlaps_flowers_rddm.tsv) results/differential_expression/v4_sex_biased_flowers_rddm.bed | cut -f 1-6 > results/candidates_flowers_clusters.bed
    
    awk 'FNR==NR{a[$0];next}{if (($4 in a)){print}}' <(cut -f 3 results/overlaps_leaves_rddm.tsv) results/differential_expression/v4_sex_biased_leaves_rddm.bed | cut -f 1-6 > results/candidates_leaves_clusters.bed

In [ ]:
# update 21-22 nt sRNA mapping BED files with the new genome using crossmap

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/females_flowers_ptgs.bed raw/depth/v4_females_flowers_ptgs.bed
sed 's/Y/chrY/g' raw/depth/v4_females_flowers_ptgs.bed > raw/depth/females_flowers_ptgs.bed

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/males_flowers_ptgs.bed raw/depth/v4_males_flowers_ptgs.bed
sed 's/Y/chrY/g' raw/depth/v4_males_flowers_ptgs.bed > raw/depth/males_flowers_ptgs.bed

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/females_leaves_ptgs.bed raw/depth/v4_females_leaves_ptgs.bed
sed 's/Y/chrY/g' raw/depth/v4_females_leaves_ptgs.bed > raw/depth/females_leaves_ptgs.bed

CrossMap.py bed --chromid a annotation/anchor_v03_v04.chain raw/depth/males_leaves_ptgs.bed raw/depth/v4_males_leaves_ptgs.bed
sed 's/Y/chrY/g' raw/depth/v4_males_leaves_ptgs.bed > raw/depth/males_leaves_ptgs.bed

rm raw/depth/v4*

---I did it for leaves but it is't necessary

In [ ]:
# Get the RPM in the gene candidates, the promoter and +-2000bp
# Since we don't have methylation data in flowers, we will only do this with the flower candidates
mkdir candidates

# make 20 windows for each gene, they will go from window 21 to 40
bedtools makewindows -b results/candidates_flowers_genes.bed -n 20 -i src | awk 'BEGIN {OFS="\t"} {print $0, ((NR - 1) % 20 + 21)}' > candidates/gene_windows.bed

# make windows for the surroundings
# first upstream, the odd lines. Then make 20 windows, from 1 to 20
bedtools flank -i results/candidates_flowers_genes.bed -g v4_chromSizes.txt -b 2000 | sed -n 'p;n' > candidates/upstream.bed
bedtools makewindows -b candidates/upstream.bed -n 20 -i src | awk 'BEGIN {OFS="\t"} {print $0, (((NR - 1) % 20) + 1)}' > candidates/upstream_windows.bed

# then downstream, the even lines. Then make 20 windows, from 41 to 60
bedtools flank -i results/candidates_flowers_genes.bed -g v4_chromSizes.txt -b 2000 | sed -n 'n;p' > candidates/downstream.bed
bedtools makewindows -b candidates/downstream.bed -n 20 -i src | awk 'BEGIN {OFS="\t"} {print $0, (((NR - 1) % 20) + 41)}' > candidates/downstream_windows.bed

# merge and transform
cat candidates/*windows.bed > candidates/full_windows.bed
bedtools sort -i candidates/full_windows.bed > candidates/windows.bed


In [ ]:
Now get the mapping data, again only flowers because there is no methylation data for leaves

######## 24 nt Mapping

bedtools intersect -a raw/depth/females_flowers_rddm.bed -b candidates/windows.bed -wb | cut -f 1,3-6,10,11 | awk '{print $0 "\t" "Female" "\t" "24"}' > candidates/females_rddm.depth
bedtools intersect -a raw/depth/males_flowers_rddm.bed -b candidates/windows.bed -wb | cut -f 1,3-6,10,11 | awk '{print $0 "\t" "Male" "\t" "24"}' > candidates/males_rddm.depth

######## 21-22 nt Mapping

bedtools intersect -a raw/depth/females_flowers_ptgs.bed -b candidates/windows.bed -wb | cut -f 1,3-6,10,11 | awk '{print $0 "\t" "Female" "\t" "21-22"}' > candidates/females_ptgs.depth
bedtools intersect -a raw/depth/males_flowers_ptgs.bed -b candidates/windows.bed -wb | cut -f 1,3-6,10,11 | awk '{print $0 "\t" "Male" "\t" "21-22"}' > candidates/males_ptgs.depth

# merge
cat candidates/*.depth > candidates/candidates_mapping.tsv

In [ ]:
# get only the TEs near the gene candidates
bedtools window -b annotation/v4_repeats.bed -a results/candidates_flowers_genes.bed -l 2200 -r 2000 | cut -f 4,7-9,12,16 | grep -v "Simple" | grep -v "Low_complexity" > results/candidates_flowers_tes.nbh

In [ ]:
# convert positions to window position, for genes, clusters and repeats
# for genes I will do it manually in R, since they all go from window 21 to 40
# include strand, for gggenes use TRUE or FALSE, where TRUE means forward, not directional will be drawn as forward

bedtools intersect -a results/candidates_flowers_clusters.bed -b candidates/windows.bed -wb | cut -f 4,6,10-11 | awk '{print $0 "\t" "cluster"}' > temp.tsv
bedtools intersect -a <(cut -f 2-6 results/candidates_flowers_tes.nbh) -b candidates/windows.bed -wb | cut -f 4,5,9-10 | awk '{print $2 "\t" $1 "\t" $3 "\t" $4 "\t" "TE" }' >> temp.tsv
sed 's/+/TRUE/g' temp.tsv | sed 's/-/FALSE/g' | sed -e 's/\t\.\t/\tTRUE\t/g' > candidates/neighbors_window.tsv
rm temp.tsv

In [ ]:
# get the fasta sequences of the candidates
perl -e ' ($id,$fasta)=@ARGV; open(ID,$id); while (<ID>) { s/\r?\n//; /^>?(\S+)/; $ids{$1}++; } $num_ids = keys %ids; open(F, $fasta); $s_read = $s_wrote = $print_it = 0; while (<F>) { if (/^>(\S+)/) { $s_read++; if ($ids{$1}) { $s_wrote++; $print_it = 1; delete $ids{$1} } else { $print_it = 0 } }; if ($print_it) { print $_ } }; END { warn "Searched $s_read FASTA records.\nFound $s_wrote IDs out of $num_ids in the ID list.\n" } ' <(cut -f 4 results/candidates_flowers_genes.bed) annotation/mrnas.fa > candidates/candidates.fa
# also the candidate from leaves
perl -e ' ($id,$fasta)=@ARGV; open(ID,$id); while (<ID>) { s/\r?\n//; /^>?(\S+)/; $ids{$1}++; } $num_ids = keys %ids; open(F, $fasta); $s_read = $s_wrote = $print_it = 0; while (<F>) { if (/^>(\S+)/) { $s_read++; if ($ids{$1}) { $s_wrote++; $print_it = 1; delete $ids{$1} } else { $print_it = 0 } }; if ($print_it) { print $_ } }; END { warn "Searched $s_read FASTA records.\nFound $s_wrote IDs out of $num_ids in the ID list.\n" } ' <(cut -f 4 results/candidates_leaves_genes.bed) annotation/mrnas.fa >> candidates/candidates.fa

In [ ]:
# then I copied the files to my local computer

No overlaps were found!

### Transcription Factor Annotation

In [ ]:
# update V3 to V4, rename the chrs cause chrY is now Y

for i in *.bed

do
name=$(basename ${i} .bed)

echo "working with ${name} "
CrossMap bed --chromid a v3_to_v4.chain ${i} v4_bad_${name}.bed
sed 's/Y/chrY/g' v4_bad_${name}.bed > v4_${name}.bed
rm v4_bad_${name}.bed 
done


In [6]:
# to get Silat TFs from the Plant TFDB, Dianthus caryophyllus

# make the BLAST db
! makeblastdb -in Dcr_pep.fas -dbtype prot 
# 1318 TF

# BLAST
! blastx -db Dcr_pep.fas -query mrnas_v3.fa -num_threads 12 -outfmt 6 -out tables/tfs.blast



Building a new DB, current time: 05/06/2024 16:17:45
New DB name:   /mnt/c/Users/eddyj/OneDrive/Escritorio/MEME/montpellier/internship/linux/Dcr_pep.fas
New DB title:  Dcr_pep.fas
Sequence type: Protein
Deleted existing Protein BLAST database named /mnt/c/Users/eddyj/OneDrive/Escritorio/MEME/montpellier/internship/linux/Dcr_pep.fas
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1318 sequences in 0.525146 seconds.


# <span style="color:#BE18D2"> Chi Square tests

Done in my personal laptop

In [4]:
# for ptgs, clean bed
! cut -f 1-4 v4_mrnas.bed > tables/v4_genes.bed

# Add -+ 2000bp to genes
! bedtools slop -i tables/v4_genes.bed -g v4_chr_sizes.tsv -b 2000 -s > tables/v4_long_genes.bed

In [5]:
# expand sex-biased gene bed files
! bedtools slop -i tables/v4_sex_biased_flowers_sbge.bed -g v4_chr_sizes.tsv -b 2000 -s > tables/v4_long_flowers_sbge.bed
! bedtools slop -i tables/v4_sex_biased_leaves_sbge.bed -g v4_chr_sizes.tsv -b 2000 -s > tables/v4_long_leaves_sbge.bed

In [6]:
# make bed file for the clusters
! tail -n +2 tables/21-22_Results.txt | awk '{print $3 "\t" $4 "\t" $5 "\t" $2 }' > tables/ptgs_clusters.bed
! tail -n +2 tables/24_Results.txt | awk '{print $3 "\t" $4 "\t" $5 "\t" $2 "_RdDM" }' > tables/rddm_clusters.bed

In [ ]:
# update coordinates to v4 genome (i did this in Sol bc I couldn't install Crossmap in my laptop)
CrossMap.py bed --chromid a v3_to_v4.chain.gz ptgs_clusters.bed v4_ptgs_clusters.bed
CrossMap.py bed --chromid a v3_to_v4.chain.gz rddm_clusters.bed v4_rddm_clusters.bed

sed 's/Y/chrY/g' v4_ptgs_clusters.bed > clean_v4_ptgs_clusters.bed
sed 's/Y/chrY/g' v4_rddm_clusters.bed > clean_v4_rddm_clusters.bed

# the files are inside tables/

In [1]:
# I made a script to avoid writting code many times, it needs the following arguments:
# 1 - full gene bed file (only 4 columns) (extended for RdDM)
# 2 - full cluster bed file (only 4 columns)
# 3 - sexb genes bed file
# 4 - sexb clusters bed file

# ptgs flowers
! bash chisq.sh tables/v4_genes.bed tables/clean_v4_ptgs_clusters.bed tables/v4_sex_biased_flowers_sbge.bed tables/v4_sex_biased_flowers_ptgs.bed

mkdir: cannot create directory ‘chisq’: File exists
Female-biased Genes with overlapping Female-biased sRNAS
2
Male-biased Genes with overlapping Female-biased sRNAS
0
Female-biased Genes with overlapping Male-biased sRNAS
1
Male-biased Genes with overlapping Male-biased sRNAS
46
______________________________________
UN-biased Genes with overlapping Female-biased sRNAS
33
UN-biased Genes with overlapping Male-biased sRNAS
38
______________________________________
Female-biased Genes with overlapping UN-biased sRNAS
360
Male-biased Genes with overlapping UN-biased sRNAS
775
______________________________________
UN-biased Genes with overlapping UN-biased sRNAS
6720
Female-biased Genes WITHOUT overlapping sRNAS
896
Male-biased Genes WITHOUT overlapping sRNAS
2752
UN-biased Genes WITHOUT overlapping sRNAS
23882


In [2]:
# ptgs leaves
! bash chisq.sh tables/v4_genes.bed tables/clean_v4_ptgs_clusters.bed tables/v4_sex_biased_leaves_sbge.bed tables/v4_sex_biased_leaves_ptgs.bed

Female-biased Genes with overlapping Female-biased sRNAS
3
Male-biased Genes with overlapping Female-biased sRNAS
0
Female-biased Genes with overlapping Male-biased sRNAS
0
Male-biased Genes with overlapping Male-biased sRNAS
32
______________________________________
UN-biased Genes with overlapping Female-biased sRNAS
33
UN-biased Genes with overlapping Male-biased sRNAS
42
______________________________________
Female-biased Genes with overlapping UN-biased sRNAS
68
Male-biased Genes with overlapping UN-biased sRNAS
7
______________________________________
UN-biased Genes with overlapping UN-biased sRNAS
7785
Female-biased Genes WITHOUT overlapping sRNAS
317
Male-biased Genes WITHOUT overlapping sRNAS
1048
UN-biased Genes WITHOUT overlapping sRNAS
26165


In [3]:
# rddm flowers
! bash chisq.sh tables/v4_long_genes.bed tables/clean_v4_rddm_clusters.bed tables/v4_sex_biased_flowers_sbge.bed tables/v4_sex_biased_flowers_rddm.bed

Female-biased Genes with overlapping Female-biased sRNAS
2
Male-biased Genes with overlapping Female-biased sRNAS
0
Female-biased Genes with overlapping Male-biased sRNAS
1
Male-biased Genes with overlapping Male-biased sRNAS
46
______________________________________
UN-biased Genes with overlapping Female-biased sRNAS
110
UN-biased Genes with overlapping Male-biased sRNAS
115
______________________________________
Female-biased Genes with overlapping UN-biased sRNAS
360
Male-biased Genes with overlapping UN-biased sRNAS
775
______________________________________
UN-biased Genes with overlapping UN-biased sRNAS
15289
Female-biased Genes WITHOUT overlapping sRNAS
896
Male-biased Genes WITHOUT overlapping sRNAS
2752
UN-biased Genes WITHOUT overlapping sRNAS
15280


In [7]:
# rddm leaves
! bash scripts/chisq.sh tables/v4_long_genes.bed tables/clean_v4_rddm_clusters.bed tables/v4_sex_biased_leaves_sbge.bed tables/v4_sex_biased_leaves_rddm.bed

Female-biased Genes with overlapping Female-biased sRNAS
3
Male-biased Genes with overlapping Female-biased sRNAS
0
Female-biased Genes with overlapping Male-biased sRNAS
0
Male-biased Genes with overlapping Male-biased sRNAS
32
______________________________________
UN-biased Genes with overlapping Female-biased sRNAS
91
UN-biased Genes with overlapping Male-biased sRNAS
102
______________________________________
Female-biased Genes with overlapping UN-biased sRNAS
68
Male-biased Genes with overlapping UN-biased sRNAS
7
______________________________________
UN-biased Genes with overlapping UN-biased sRNAS
17327
Female-biased Genes WITHOUT overlapping sRNAS
317
Male-biased Genes WITHOUT overlapping sRNAS
1048
UN-biased Genes WITHOUT overlapping sRNAS
16599


## Manual Chisqr test data

In [20]:
# somehow, later use of my script with DMRs was underestimating the real numbers
! rm -R chisq
! mkdir chisq

# create bed of unbiased genes
! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_flowers_sbge.bed ) tables/v4_genes.bed | cut -f 1-4 > chisq/unbiased.genes_flowers.bed
! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_leaves_sbge.bed ) tables/v4_genes.bed | cut -f 1-4 > chisq/unbiased.genes_leaves.bed

! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_flowers_sbge.bed ) tables/v4_long_genes.bed | cut -f 1-4 > chisq/unbiased.long_genes_flowers.bed
! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_leaves_sbge.bed ) tables/v4_long_genes.bed | cut -f 1-4 > chisq/unbiased.long_genes_leaves.bed


# create list of unbiased clusters
! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_flowers_ptgs.bed ) tables/clean_v4_ptgs_clusters.bed | cut -f 1-4 > chisq/unbiased.clusters_flowers_ptgs.bed
! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_flowers_rddm.bed ) tables/clean_v4_rddm_clusters.bed | cut -f 1-4 > chisq/unbiased.clusters_flowers_rddm.bed

! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_leaves_ptgs.bed ) tables/clean_v4_ptgs_clusters.bed | cut -f 1-4 > chisq/unbiased.clusters_leaves_ptgs.bed
! awk 'FNR==NR{a[$0];next}{if (!($4 in a)){print}}' <(cut -f 4 tables/v4_sex_biased_leaves_rddm.bed ) tables/clean_v4_rddm_clusters.bed | cut -f 1-4 > chisq/unbiased.clusters_leaves_rddm.bed

# check outputs
! ls -l chisq/ # all different, good!

total 11748
-rwxrwxrwx 1 somnya somnya 1630338 Jul 24 15:52 unbiased.clusters_flowers_ptgs.bed
-rwxrwxrwx 1 somnya somnya 1849267 Jul 24 15:52 unbiased.clusters_flowers_rddm.bed
-rwxrwxrwx 1 somnya somnya 1641514 Jul 24 15:52 unbiased.clusters_leaves_ptgs.bed
-rwxrwxrwx 1 somnya somnya 1861883 Jul 24 15:52 unbiased.clusters_leaves_rddm.bed
-rwxrwxrwx 1 somnya somnya 1192588 Jul 24 15:52 unbiased.genes_flowers.bed
-rwxrwxrwx 1 somnya somnya 1322391 Jul 24 15:52 unbiased.genes_leaves.bed
-rwxrwxrwx 1 somnya somnya 1192588 Jul 24 15:52 unbiased.long_genes_flowers.bed
-rwxrwxrwx 1 somnya somnya 1322390 Jul 24 15:52 unbiased.long_genes_leaves.bed


In [19]:
# Flowers PTGS
# in order: fem genes, male genes, unbiased genes
# all are counts, genes overlapping with multiple clusters are removed

# female-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_sex_biased_flowers_sbge.bed) -b <(grep "female" tables/v4_sex_biased_flowers_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_sex_biased_flowers_sbge.bed) -b <(grep "female" tables/v4_sex_biased_flowers_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.genes_flowers.bed -b <(grep "female" tables/v4_sex_biased_flowers_ptgs.bed) | cut -f 4 | uniq | wc -l

# male-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_sex_biased_flowers_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_flowers_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_sex_biased_flowers_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_flowers_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.genes_flowers.bed -b <(grep -v "female" tables/v4_sex_biased_flowers_ptgs.bed) | cut -f 4 | uniq | wc -l

# unbiased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_sex_biased_flowers_sbge.bed) -b chisq/unbiased.clusters_flowers_ptgs.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_sex_biased_flowers_sbge.bed) -b chisq/unbiased.clusters_flowers_ptgs.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.genes_flowers.bed -b chisq/unbiased.clusters_flowers_ptgs.bed | cut -f 4 | uniq | wc -l

2
0
33
1
46
38
360
775
6720


In [23]:
# Leaves PTGS
# female-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_sex_biased_leaves_sbge.bed) -b <(grep "female" tables/v4_sex_biased_leaves_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_sex_biased_leaves_sbge.bed) -b <(grep "female" tables/v4_sex_biased_leaves_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.genes_leaves.bed -b <(grep "female" tables/v4_sex_biased_leaves_ptgs.bed) | cut -f 4 | uniq | wc -l

# male-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_sex_biased_leaves_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_leaves_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_sex_biased_leaves_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_leaves_ptgs.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.genes_leaves.bed -b <(grep -v "female" tables/v4_sex_biased_leaves_ptgs.bed) | cut -f 4 | uniq | wc -l

# unbiased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_sex_biased_leaves_sbge.bed) -b chisq/unbiased.clusters_leaves_ptgs.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_sex_biased_leaves_sbge.bed) -b chisq/unbiased.clusters_leaves_ptgs.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.genes_leaves.bed -b chisq/unbiased.clusters_leaves_ptgs.bed | cut -f 4 | uniq | wc -l

3
0
33
0
32
42
68
7
7785


In [25]:
# Flowers RdDM

# female-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_long_flowers_sbge.bed) -b <(grep "female" tables/v4_sex_biased_flowers_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_long_flowers_sbge.bed) -b <(grep "female" tables/v4_sex_biased_flowers_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.long_genes_flowers.bed -b <(grep "female" tables/v4_sex_biased_flowers_rddm.bed) | cut -f 4 | uniq | wc -l

# male-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_long_flowers_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_flowers_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_long_flowers_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_flowers_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.long_genes_flowers.bed -b <(grep -v "female" tables/v4_sex_biased_flowers_rddm.bed) | cut -f 4 | uniq | wc -l

# unbiased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_long_flowers_sbge.bed) -b chisq/unbiased.clusters_flowers_rddm.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_long_flowers_sbge.bed) -b chisq/unbiased.clusters_flowers_rddm.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.long_genes_flowers.bed -b chisq/unbiased.clusters_flowers_rddm.bed | cut -f 4 | uniq | wc -l

9
7
110
2
107
115
770
1445
15289


In [26]:
# Leaves RdDM

# female-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_long_leaves_sbge.bed) -b <(grep "female" tables/v4_sex_biased_leaves_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_long_leaves_sbge.bed) -b <(grep "female" tables/v4_sex_biased_leaves_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.long_genes_leaves.bed -b <(grep "female" tables/v4_sex_biased_leaves_rddm.bed) | cut -f 4 | uniq | wc -l

# male-biased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_long_leaves_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_leaves_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_long_leaves_sbge.bed) -b <(grep -v "female" tables/v4_sex_biased_leaves_rddm.bed) | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.long_genes_leaves.bed -b <(grep -v "female" tables/v4_sex_biased_leaves_rddm.bed) | cut -f 4 | uniq | wc -l

# unbiased clusters
! bedtools intersect -nonamecheck -a <(grep "female" tables/v4_long_leaves_sbge.bed) -b chisq/unbiased.clusters_leaves_rddm.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a <(grep -v "female" tables/v4_long_leaves_sbge.bed) -b chisq/unbiased.clusters_leaves_rddm.bed | cut -f 4 | uniq | wc -l
! bedtools intersect -nonamecheck -a chisq/unbiased.long_genes_leaves.bed -b chisq/unbiased.clusters_leaves_rddm.bed | cut -f 4 | uniq | wc -l

7
0
91
1
86
102
164
30
17327


# sRNA mapping in the Y chromosome

In [ ]:
# check the name of the chromosomes
! grep "Y" epi/males_flowers_ptgs.bed | head

# get genes on the Y and rename Y to "chrY"
! grep "Y" paper_revisions/v4_genes.bed | sed 's/Y/chrY/g' > paper_revisions/v4_y_genes.bed
! grep "Y" paper_revisions/v4_long_genes.bed | sed 's/Y/chrY/g' > paper_revisions/v4_y_long_genes.bed

# create input files for surco
# only males of course

#### 21-22 nt
! ml bedtools2/2.31.1; bedtools intersect -nonamecheck -a epi/males_flowers_ptgs.bed -b paper_revisions/v4_y_genes.bed -wb | cut -f 1-6,10 > paper_revisions/y_males_flowers_ptgs.tsv
! ml bedtools2/2.31.1; bedtools intersect -nonamecheck -a epi/males_leaves_ptgs.bed -b paper_revisions/v4_y_genes.bed -wb | cut -f 1-6,10 > paper_revisions/y_males_leaves_ptgs.tsv

##### 24 nt
! ml bedtools2/2.31.1; bedtools intersect -nonamecheck -a epi/males_flowers_rddm.bed -b paper_revisions/v4_y_long_genes.bed -wb | cut -f 1-6,10 > paper_revisions/y_males_flowers_rddm.tsv
! ml bedtools2/2.31.1; bedtools intersect -nonamecheck -a epi/males_leaves_rddm.bed -b paper_revisions/v4_y_long_genes.bed -wb | cut -f 1-6,10 > paper_revisions/y_males_leaves_rddm.tsv